PCA - from scratch

In [2]:
SEED = 42

In [3]:
import pandas as pd
df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/'
                      'machine-learning-databases/wine/wine.data',
                      header=None)
df_wine.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash',
                   'Alcalinity of ash', 'Magnesium', 'Total phenols',
                   'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
                   'Color intensity', 'Hue',
                   'OD280/OD315 of diluted wines', 'Proline']

df_wine.head()

,Class label,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [4]:
from sklearn.model_selection import train_test_split

X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.3, 
                     stratify=y,
                     random_state=SEED)

In [5]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

In [9]:
#Obtrain eigenpairs covariance matrix
import numpy as np
cov_mat = np.cov(X_train_std.T)
eigen_vals, eigen_vecs = np.linalg.eig(cov_mat)
eigen_vals
eigen_vecs

array([[ 0.15763477,  0.49414675,  0.13550025, -0.22002101, -0.1001375 ,
         0.20436914,  0.00578955,  0.04666841, -0.29868842, -0.56269774,
        -0.17889091, -0.38924236, -0.14436028],
       [-0.25237011,  0.1670862 , -0.08484512,  0.34863385, -0.51191077,
         0.36452972,  0.51124855,  0.03381991,  0.22155089, -0.06054546,
         0.17580919,  0.11403549, -0.15978215],
       [-0.01662626,  0.33581955, -0.61436922, -0.21029079,  0.10967198,
         0.18888033, -0.0707048 ,  0.17376559,  0.02928534,  0.18377711,
        -0.44222618,  0.37290998, -0.11455204],
       [-0.23514225, -0.0230456 , -0.62199483,  0.11850515,  0.01880619,
         0.03226373, -0.34711322, -0.16492916, -0.04198251, -0.3281213 ,
         0.44478733, -0.19926928,  0.21726025],
       [ 0.14285062,  0.2816766 , -0.08744449,  0.54616385,  0.60714326,
         0.03967105,  0.24673831, -0.08350982,  0.03735091,  0.19114731,
        -0.02153713, -0.33780751, -0.06123678],
       [ 0.38989121,  0.071647